In [66]:
import re

#Step 1: Replace 1st person with 2nd person references
def replace_pronouns(text):
  replacements = { r"\bI( 'm| am)\b":"You are",
                   r"\bI\b":"You",
                   r"\bI have\b":"You have",
                   r"\bmy\b":"your",
                   r"\bme\b":"you"}
  for pattern, replace_with in replacements.items():
    text = re.sub(pattern, replace_with , text, flags = re.IGNORECASE)
  return text


def generate_without_rank(text):
  patterns = [
          (r"all(.*)are(.*)", ["It sounds like you think all \g<1> are \g<2> How does that make you feel?"]),
          (r"feel like(.*)always(.*)", ["You feel like \g<1> always \g<2> Can you please be more specific?"]),
          (r"feel like(.*?)never stops", ["You feel like \g<1> never stops. That sounds exhausting."]),
          (r"it makes (.*)", ["I am sorry to hear that it makes \g<1>"]),
          (r"sometimes(.*)", ["You sometimes \g<1>"]),
          (r"have tried many(.*)", ["You have tried many \g<1> Can you be more specific?"]),
          (r"have tried(.*)", ["Is \g<1> helping?"]),
          (r"assist(\s)(\w+)(\s)with(.*)", ["Me assisting \g<2> with \g<4>"])
      ]
  for pattern, responses in patterns:
    match = re.search(pattern, text, flags=re.IGNORECASE)
    if match:
      response_template = responses[0]  # Choose the first response by default
      response = response_template

      for i in range(1, len(match.groups()) + 1):
        response = response.replace(f"\\g<{i}>", match.group(i).strip())
      return response

  return "Tell me more"




#Step 2: Use additional re to generate response
#Using patterns with capture groups
#Step 3: Use scores to rank possible transformations
def generate_response(text):
    patterns = [
        (r"all(.*)are(.*)", "It sounds like you think all \g<1> are \g<2> How does that make you feel?", 6),
        (r"feel like(.*?)\s*always\s*(.*)", "You feel like \g<1> always \g<2> Can you please be more specific?", 4),
        (r"feel like\s*(.*?)\s*never stops", "You feel like \g<1> never stops. That sounds exhausting.", 5),
        (r"it makes (.*)", "I am sorry to hear that it makes \g<1>", 7),
        (r"sometimes\s*(.*)", "You sometimes \g<1>", 4),
        (r"have tried many\s*(.*)", "You have tried many \g<1> Can you be more specific?", 3),
        (r"have tried(.*)", "Is \g<1> helping?", 4),
        (r"assist(\s)(\w+)(\s)with(.*)", "Me assisting \g<2> with \g<4>", 4)
    ]
    matched_responses = []

    # Loop through patterns
    for pattern, response, score in patterns:
      #Use re.search instead of re.match to check for matches anywhere in the string, not just at the start
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            substituted_response = response
            # Manually substitute the captured groups in the response
            for i in range(1, match.lastindex + 1):
                substituted_response = substituted_response.replace(f"\\g<{i}>", match.group(i))

            matched_responses.append((substituted_response, score))

    # Return the highest scored match
    if matched_responses:
        best_response = sorted(matched_responses, key=lambda x: x[1], reverse=True)[0][0]
        return best_response

    # Default response if no match
    return "Tell me more"

def main():
  print("CLIENT: ", end="")
  clientResponse = input()

  while True:
    match = re.match(r"Eliza(.*)[Ss]top(.*)", clientResponse)

    if match:
      print("ElIZA: Goodbye!")
      break
    else:

      print("ELIZA (ranked):",generate_response(replace_pronouns(clientResponse)))
      print("ELIZA (unranked):",generate_without_rank(replace_pronouns(clientResponse)))
      print()

      print("CLIENT: ", end="")
      clientResponse = input()

if __name__ == "__main__":
    main()

Client: I feel like there is always so much work.
ELIZA (ranked): You feel like  there is always so much work. Can you please be more specific?
ELIZA (unranked): You feel like there is always so much work. Can you please be more specific?

Client: All four modules; COMP306, COMP313, COMP315, and COMP316 are overwhelming me.
ELIZA (ranked): It sounds like you think all  four modules; COMP306, COMP313, COMP315, and COMP316  are  overwhelming you. How does that make you feel?
ELIZA (unranked): It sounds like you think all four modules; COMP306, COMP313, COMP315, and COMP316 are overwhelming you. How does that make you feel?

Client: It makes me anxious.
ELIZA (ranked): I am sorry to hear that it makes you anxious.
ELIZA (unranked): I am sorry to hear that it makes you anxious.

Client: I sometimes sit there staring at my screen, not even knowing where to begin.
ELIZA (ranked): You sometimes sit there staring at your screen, not even knowing where to begin.
ELIZA (unranked): You sometimes 